In [1]:
import time
import random
import math

In [2]:
pessoas = [("Amanda", "CWB"),
           ("Pedro", "GIG"),
           ("Marcos", "POA"),
           ("Priscila", "FLN"),
           ("Jessica", "CNF"),
           ("Paulo", "GYN"),]

In [3]:
destino = "GRU"

In [4]:
voos = {}

In [5]:
with open("voos.txt", "r") as arq:
    for linha in arq.readlines():
        origem, destino, saida, chegada, preco = linha.split(",")
        voos.setdefault((origem, destino), [])
        voos[(origem, destino)].append((saida, chegada, int(preco)))

In [6]:
# [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

def imprimir_agenda(agenda: list[int]) -> None:
    id_voo = -1
    for i in range(len(agenda)//2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]
        print(f"{nome} {origem} {ida[0]} {ida[1]} {ida[2]} {volta[0]} {volta[1]} {volta[2]}\n")

In [7]:
agenda = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

imprimir_agenda(agenda)

Amanda CWB 8:04 10:11 95 12:08 14:05 142

Pedro GIG 10:30 14:57 290 9:49 13:51 229

Marcos POA 17:08 19:08 262 10:32 13:16 139

Priscila FLN 15:34 18:11 326 11:08 14:38 262

Jessica CNF 9:42 11:32 169 12:08 14:47 231

Paulo GYN 13:37 15:08 250 11:07 13:24 171



In [12]:
def obter_minutos(horaStr: str):
    hora = time.strptime(horaStr, "%H:%M")
    minutos = hora[3] * 60 + hora[4]
    return minutos

In [18]:
def custo(solucao) -> None:
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = obter_minutos("23:59")

    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]

        preco_total += ida[2] + volta[2]

        if ultima_chegada < obter_minutos(ida[1]):
            ultima_chegada = obter_minutos(ida[1])

        if primeira_partida > obter_minutos(volta[0]):
            primeira_partida = obter_minutos(volta[0])
    
    total_espera = 0
    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]

        total_espera += (ultima_chegada - obter_minutos(ida[1])) + (obter_minutos(volta[0]) - primeira_partida)

    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera 

In [19]:
custo(agenda)

4635